<a href="https://colab.research.google.com/github/trainocate-japan/extending_genai_with_langchain/blob/main/chapter4/exercise2/exercise2_solution.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 演習の準備
---

## 必要なライブラリのインストール

In [ ]:
!pip install -q langchain langchain-openai langchain-community langchain-core langchain-pinecone pypdf docx2txt wikipedia pinecone

In [ ]:
# !pip install -q langchain==0.2.15 langchain-openai==0.1.23 langchain-community==0.2.15 langchain-core==0.2.37 langchain-text-splitters==0.2.2

In [ ]:
!pip freeze | grep langchain

## API キーの設定

*  左ナビゲーションで [**シークレット**] アイコン (鍵形のアイコン) をクリックします。
*  [**新しいシークレットを追加**] をクリックし、`LANGCHAIN_API_KEY`、`OPENAI_API_KEY`、`PINECONE_API_KEY` の 3 つを設定し、[**ノートブックからのアクセス**] を有効にします。
  *  `OPENAI_API_KEY` の [**値**] には指定されたキーを入力します。
  *  `LANGCHAIN_API_KEY` と `PINECONE_API_KEY` の [**値**] にはご自身で取得したキーを入力してください。
*  入力が完了したら、下のセルを実行します。

In [ ]:
import os
from google.colab import userdata

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "default"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get('LANGCHAIN_API_KEY')

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')

## サンプルファイルのアップロード

*  左ナビゲーションで [**ファイル**] アイコンをクリックします。
*  [sample_data] 下の何もない部分で右クリックし、[**新しいフォルダ**] をクリックします。
*  作成されたフォルダに **files** という名前を設定します。
*  files フォルダにカーソルを合わせ、3 点リーダアイコンをクリックして、[**アップロード**] をクリックします。
*  ローカルの files フォルダにあるすべてのファイルを選択してアップロードします。
*  「警告」のポップアップが表示されますが問題ありません。[**OK**] をクリックしてポップアップを閉じます。

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# Section 1: 関数の作成
---
RAG の各処理を実行する関数を作成します。  
タスク (Task) になっている各セルのコードの不足している部分を補完して処理を実装してください。

## Task: ドキュメントをロードする関数の作成
Python と LangChain フレームワークを使用して、PDF、Word (.docx)、およびテキスト (.txt) の各ドキュメントフォーマットに応じて動的にドキュメントをロードする関数を作成します。  
ファイル拡張子に基づいて適切な Document loader を選択し、対応するライブラリを使用してドキュメントをロードするコードを書いてください。  

* ファイル名から拡張子を抽出し、それに基づいて適切な Document loader を使用します
* 他の拡張子の場合は、対応していない旨のメッセージを表示して、None を返します
* 各ファイル形式に対応する Document loader を動的にインポートするコードを記述します。必要なインポートは、条件分岐の中で行います
* ロードプロセス中の状況を示すために、処理中のファイル名を `print` で表示します
* ドキュメントの内容をロードし、`data` 変数に格納して関数の出力として返します
  
参考：  
[langchain_community.document_loaders.text.TextLoader](https://python.langchain.com/v0.2/api_reference/community/document_loaders/langchain_community.document_loaders.text.TextLoader.html)  
[langchain_community.document_loaders.pdf.PyPDFLoader](https://python.langchain.com/v0.2/api_reference/community/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)  
[langchain_community.document_loaders.word_document.Docx2txtLoader](https://python.langchain.com/v0.2/api_reference/community/document_loaders/langchain_community.document_loaders.word_document.Docx2txtLoader.html)  
[langchain_community.document_loaders.wikipedia.WikipediaLoader](https://python.langchain.com/v0.2/api_reference/community/document_loaders/langchain_community.document_loaders.wikipedia.WikipediaLoader.html)  
https://python.langchain.com/v0.2/docs/integrations/document_loaders/

In [ ]:
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain_community.document_loaders.pdf import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain_community.document_loaders.word_document import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    elif extension == '.txt':
        from langchain_community.document_loaders.text import TextLoader
        loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data

## Task: ドキュメントをチャンクに分割する関数の作成
ロードしたドキュメント データを指定されたサイズにチャンク分割する関数を作成します。  

* Text splitter には `RecursiveCharacterTextSplitter` クラスを使用します
* `chunk_size` パラメータを使用して、各チャンクのサイズを指定します
* `chunk_overlap` パラメータを使用して、隣接するチャンク間の重複サイズを設定します
* これらのパラメータは関数の引数として指定でき、デフォルト値はそれぞれ `256` と `0` です
* インスタンス化した `RecursiveCharacterTextSplitter` を使用して、入力されたドキュメント データを分割します
* 関数は、分割されたチャンクを `chunks` というリストとして返します。
  
参考：  
[langchain_text_splitters.character.RecursiveCharacterTextSplitter](https://python.langchain.com/v0.2/api_reference/text_splitters/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)

In [ ]:
def chunk_data(data, chunk_size=256, chunk_overlap=0):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = text_splitter.split_documents(data)
    return chunks

## Task: Embedding を行い Vector store インスタンスを出力する関数の作成
ドキュメントのチャンクの embeddings を Pinecone の Index に挿入するか、既存の Index から embeddings を取得する関数を作成します。Index の存在チェック、インデックスの作成、embeddings の生成および挿入の処理を実装します。

* この関数は引数として、Index 名 (`index_name`) とチャンクのリスト (`chunks`) を受け取ります
* Vector store には Pinecone を使用します
* Embedding model には OpenAI の `text-embedding-3-small` を使用します。embedding のベクトル次元数は `1536` です
* Pinecone クライアントの `list_indexes()` メソッドを使用して、指定された `index_name` が既に存在するかどうかを確認します
* Index が存在する場合は、既存の Index から Pinecone の Vector store インスタンスを作成します
* Index が存在しない場合は、新しい Index を作成します。create_index メソッドを使用して、指定された次元数 (`1536`) とコサイン類似度 (`cosine`) を使用して Index を作成します
* チャンクの embeddings を生成し、それらを新しい Index に挿入し、Pinecone の Vector store インスタンスを作成します
* 最後に、Vector store インスタンス `vector_store` を関数の出力として返します

参考：  
[langchain_openai.embeddings.base.OpenAIEmbeddings](https://python.langchain.com/v0.2/api_reference/openai/embeddings/langchain_openai.embeddings.base.OpenAIEmbeddings.html)  
[langchain_pinecone.vectorstores.PineconeVectorStore](https://python.langchain.com/v0.2/api_reference/pinecone/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html#pineconevectorstore)  
https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/  


In [ ]:
def insert_or_fetch_embeddings(index_name, chunks):
    # 必要なライブラリをインポート
    import pinecone
    from langchain_pinecone import PineconeVectorStore
    from langchain_openai import OpenAIEmbeddings
    from pinecone import PodSpec, ServerlessSpec

    # Pinecone クライアントを初期化
    pc = pinecone.Pinecone()

    # Embedding model のインスタンスを作成
    embedding_model = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)

    # embeddings の作成/ロード、Vector store の作成
    if index_name in pc.list_indexes().names():
        # Index がすでに存在する場合
        print(f'Index {index_name} already exists. Loading embeddings ... ')
        # Vector store インスタンスを作成
        vector_store = Pinecone.from_existing_index(index_name, embedding_model)
        print('Ok')
    else:
        # Index が存在しない場合
        print(f'Creating index {index_name} and embeddings ... ')

        # Index を作成
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-east-1'
            )
        )

        # Vector store インスタンスを作成
        vector_store = PineconeVectorStore.from_documents(chunks, embedding_model, index_name=index_name)
        print('Ok')

    return vector_store

### Index を削除する関数

In [ ]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()

    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

## Task: RAG Chain を実行する関数
---
会話履歴を考慮してユーザーの質問に回答する Chain を実行する関数を作成します。  
このタスクでは、Retriever の作成、会話履歴を考慮した質問の再構成、そして質問に対する適切な応答を生成する Chain を作成します。  

* この関数は引数として、`vector_store` 、ユーザーの質問 `q` 、Chat history のインスタンス `chat_history` 、セッション ID `session_id` (デフォルト値は `unused`) 、検索結果ドキュメントの取得数 `k` (デフォルト値は `20`) を受け取ります
* Chat model には OpenAI の `gpt-4o-mini` を使用します
* Retriever にも会話履歴を考慮させます
* LLM からの回答はテキストの形式に変換します
* `RunnableWithMessageHistory` を使用して Chain の処理に会話履歴を組み込みます
* Chain 処理の出力 `answer` を関数の出力として返します

参考：  
https://python.langchain.com/v0.2/docs/concepts/#retrievers  


In [ ]:
def get_answer_with_history(vector_store, q, chat_history, session_id='unused', k=20):
    from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
    from langchain_core.runnables import RunnablePassthrough, RunnableParallel
    from langchain_openai import ChatOpenAI
    from operator import itemgetter
    from langchain_core.output_parsers import StrOutputParser
    from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
    from langchain_core.chat_history import BaseChatMessageHistory
    from langchain_core.runnables.history import RunnableWithMessageHistory
    from langchain.chains import create_history_aware_retriever

    # Chat model
    model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

    # Retriever
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})

    # 検索用にクエリを書き換えるためのプロンプト
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, "
        "just reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_q_system_prompt),
            MessagesPlaceholder("chat_history"),
            ("human", "{input}"),
        ]
    )

    # 会話履歴を考慮する Retriever を作成
    history_aware_retriever = create_history_aware_retriever(
        model, retriever, contextualize_q_prompt
    )

    # ユーザーのクエリに回答させるための Prompt template
    system_message = """以下の参考用のテキストの一部を参照して、質問に回答してください。もし参考用のテキストの中に回答に役立つ情報が含まれていなければ、分からない、と答えてください。

    {context}"""
    human_message = "質問：{input}"

    chat_prompt = ChatPromptTemplate.from_messages([

        (
            "system", system_message
        ),
        MessagesPlaceholder("chat_history"),
        (
            "human", human_message
        )
    ])

    # ユーザーのクエリと history_aware_retriever が取得した Documents を出力する Runnable
    add_context = RunnablePassthrough.assign(context=history_aware_retriever)

    # Chain を定義
    rag_chain = add_context | chat_prompt | model | StrOutputParser()

    runnable_with_history = RunnableWithMessageHistory(
        rag_chain,
        lambda session_id: chat_history, # session_id を受け取って対応する chat message history インスタンス (BaseChatMessageHistory) を返す関数
        input_messages_key="input",
        history_messages_key="chat_history",
    )

    # Chain の実行
    answer = runnable_with_history.invoke({'input': q}, config={"configurable": {"session_id": session_id}})
    return answer

# Section 2: 関数を使用した処理の実行
---
Section 1 で作成した関数を使用して RAG の実行処理を実装します。
タスク (Task) になっている各セルのコードの不足している部分を補完して処理を実装してください。

## Task: ドキュメントのロードとチャンク化
* ファイルからドキュメントのデータをロードし、それをチャンクに分割します
* チャンクのサイズは `300` 、チャンク間の重複サイズは `0` とします

In [ ]:
# ドキュメント データをロード
data = load_document('files/employment_regulations.pdf')

# チャンクに分割
chunks = chunk_data(data, chunk_size=300, chunk_overlap=0)

# 確認のため、チャンク数を表示する
print(len(chunks))

## 既存の Index を削除

In [ ]:
delete_pinecone_index()

## Task: Vector store インスタンスを取得
Index 名を指定して、その Vector store インスタンスを取得します

In [ ]:
# Index 名を指定
index_name = 'askadocument'

# Vector store インスタンスを取得
vector_store = insert_or_fetch_embeddings(index_name=index_name, chunks=chunks)

## Task: チャットボットを実行
* Chat history には `ChatMessageHistory` を使用します
* 必要な引数を `get_answer_with_history` 関数に渡して実行し、回答を表示します

In [ ]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory

# Chat history
chat_history = ChatMessageHistory()

import time
i = 1
print('Write Quit or Exit to quit.')
while True:
    q = input(f'Question #{i}: ')
    i = i + 1
    if q.lower() in ['quit', 'exit']:
        print('Quitting ... bye bye!')
        time.sleep(2)
        break

    answer = get_answer_with_history(vector_store, q, chat_history, 'a123')
    print(f'\nAnswer: {answer}')
    print(f'\n {"-" * 50} \n')